# Sorting

Sorting algorithms come up so frequently that they deserve their own section.

In general, we want to solve the problem:

> Given an array of unsorted values, how can we sort them in ascending order?


## 1. Bubble Sort

1. Start with 2 pointers pointing at the first two values in the array. 
2. Compare the left and right values. If `left_value > right_value`, swap them. Otherwise, do nothing.
3. Move both pointers one cell rightwards.
4. Repeat Steps 2-3 until we reach values that have already been sorted. This completes the first "pass-through" and means we have "bubbled up" the biggest number to the end of the array.
5. Start over. repeat Steps 1-4 to bubble up the second biggest number into the second to last position. Repeat this until we perform a pass through with no swaps.

Each pass through loops through one fewer element:

Pass-through number | Number of operations
---------|--------------
1  | N-1
2  | N-2
3  | N-3
4  | N-4
5  | N-5
...| ...
k  | N-k

So in total, there are $(N-1) + (N-2) + (N-3) + ... + 1$ comparisons. This is the sum of an arithmetic progression, which we can calculate as $\frac{N^2}{2}$.

Also worth noting that in the worst case - an input array in descending order - each comparison will also result in a swap. This does not affect the Big-O complexity but would slow down the run time in practice.

The complexity is therefore **$O(N^2)$**.

In general, any nested loop should be a hint at quadratic time complexity.

In [49]:
input_array = [8, 7, 5, 3, 1, 9, 0, 8, 23, 69, 420, 12]

In [56]:
bubble_sort(input_array)

[0, 1, 3, 5, 7, 8, 8, 9, 12, 23, 69, 420]

## 2. Selection Sort